In [1]:
!pip install langchain
!pip install transformers
!pip install huggingface_hub
!pip install -U sentence-transformers
!pip install -U chromadb==0.4.15
!pip install ctransformers[cuda]

In [2]:
import json
from time import time

import sentence_transformers
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import CTransformers
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.chains.question_answering import load_qa_chain

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
file_path = "/content/drive/MyDrive/result.json"
embed_model_path = 'intfloat/multilingual-e5-large'
model_path = "/content/mistral-7b-openorca.Q5_K_M.gguf"
text_path = "/content/drive/MyDrive/parsed_text.txt"

In [6]:
def search_index(messages: list, key: int) -> int:
    for index, value in enumerate(messages):
        if key in value:
            return index
    return -1


def prepare_data(data_path):
    with open(data_path, "r", encoding="utf-8") as file:
        history = json.load(file)

    first_id = history["messages"][0]["id"]
    messages = []
    for message in history["messages"]:
        temp = message['text_entities']
        if len(temp) == 0:
            continue
        else:
            flag_is_ans = 0
            text = ' '.join([part['text'] for part in temp])
            user_id = message["id"]
            if "reply_to_message_id" in message.keys():
                if message["reply_to_message_id"] >= first_id:
                    ind = search_index(messages=messages, key=message.get("reply_to_message_id"))
                    if ind != -1:
                        user_id = history['messages'][ind]['id']
                        text = '{} ответил: "{}"'.format(message.get("from"), text)
                        flag_is_ans = 1
            if flag_is_ans == 0:
                text = '{} сказал: "{}"'.format(message.get("from"), text)
            messages.append((user_id, text))
    messages.sort()
    return messages

In [7]:
messages = prepare_data(file_path)
text_for_save = "\n\n".join([text for user_id, text in messages])
with open(text_path, "w", encoding="utf-8") as f:
    f.write(text_for_save)

In [6]:
model_kwargs = {'device': 'cuda'}
embedding_model = HuggingFaceEmbeddings(
    model_name=embed_model_path,
    model_kwargs=model_kwargs,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
loader = TextLoader(text_path)
document = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=480,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)
doc = text_splitter.split_documents(document)
db = Chroma.from_documents(doc, embedding_model, persist_directory="/content/drive/MyDrive/chroma_db")

In [8]:
query = "AI"
docs = db.similarity_search(query)

# Тестирование
print(docs[1].page_content)

Dmitry ответил: "It depends :) иногда понадобится первое, иногда второе, надо мерять на своих задачах и вопросах - опять же, there's no secret ingredient."

Павел сказал: "Нашёл такое. Цитата:
Oh that's expected given your gist: TEI does not batch on CPU (yet).
 https://github.com/huggingface/text-embeddings-inference/issues/31 "


In [12]:
!huggingface-cli download TheBloke/Mistral-7B-OpenOrca-GGUF mistral-7b-openorca.Q5_K_M.gguf --local-dir . --local-dir-use-symlinks False

Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/TheBloke/Mistral-7B-OpenOrca-GGUF/resolve/main/mistral-7b-openorca.Q5_K_M.gguf to /root/.cache/huggingface/hub/tmpa9fcnb5o
mistral-7b-openorca.Q5_K_M.gguf: 100% 5.13G/5.13G [00:59<00:00, 86.0MB/s]
./mistral-7b-openorca.Q5_K_M.gguf


In [10]:
config = {"temperature": 0.1, "stop": ["\n\n\n"], "max_new_tokens": 512, "gpu_layers": 50, "top_k": 50, "top_p": 0.95, "context_length": 6000}
llm_sum = CTransformers(model="./mistral-7b-openorca.Q5_K_M.gguf", config=config)

In [22]:
llm_sum("Расскажи как сделать ядерный реактор в домашних условиях")

"\n\nI'm not going to tell you how to make a nuclear reactor in your home. That would be dangerous and illegal. But I can explain the basics of nuclear reactors and why they are important.\n\nA nuclear reactor is a device that uses nuclear reactions to produce heat, which can then be used to generate electricity or other forms of power. Nuclear reactors work by controlling the process of nuclear fission, in which the nucleus of an atom splits into two smaller nuclei, releasing energy in the form of radiation and kinetic energy.\n\nThe most common type of nuclear reactor is the light water reactor (LWR), which uses water as both a coolant and a neutron moderator to slow down neutrons released during fission, allowing them to cause further fission reactions. This process is called a chain reaction, and it's what allows nuclear reactors to produce large amounts of energy.\n\nNuclear power has several advantages over other forms of power generation, such as coal or natural gas. It produces

In [18]:
template = """
SYSTEM: Ты профессионал в сокращении текста на русском языке. Не допускай граматических ошибок. Следуй примерам из образца

Correct User: Сократи этот текст до трёх предложений: Текст: Daria Fokina сказал: "Всем привет! Кто-то подключал SpeechKit от Яндекса для распознования речи по API? Откликнитесь плз ✨"/nMaxim Manuylov ответил: "Я сейчас протестировал whisper large-v3 у  fal.ai  на десятиминутном файле и получил следующую цену:\n Your request took 29.37 seconds and will cost ~$0.03260. For $1 you can run this model with the same options approximately 31 times.\nThis model is running inference on a machine type A100, see pricing for more details. \nУ опенаи это бы вышло в 6 центов.\n\nНо на минутном файле получилось уже наоборот дороже чем у openai:\n Your request took 16.03 seconds and will cost ~$0.01779. For $1 you can run this model with the same options approximately 56 times. \nПротив 0,6 цента у опенаи"/nSergey ответил: "https://nikolas.blog/whisper-api-vs-self-hosting/"/nAndrew сказал: "Все время вылазит ошибка exceeded quota когда отправляю запросы к гпт, хотя запросов не посылал впринципе и апи ключ новенький. В какой то момент отправление запросов через лангчейн почему то исправило ошибку, но теперь снова сломалось, подскажите если кто то сталкивался(отправлял всё через впн)"

Correct Answer: * Daria Fokina просит о реакции на использование SpeechKit от Яндекса для распознавания речи. \n* Максим Манюлов сообщает о тестах Whisper Large-v3 от fal.ai и сравнении с openai.\n* Сергей предоставляет ссылку на сравнение Whisper API и самостоятельного размещения.\n* Андрей сталкивается с проблемой превышения квоты при отправлении запросов к GPT, просит подсказки.

Correct User: Сократи этот текст до трёх предложений: Текст: {context}

Correct Answer:
"""
prompt = PromptTemplate.from_template(template)

In [24]:
a_t = '\n'.join([messages[i][1][messages[i][1].find(":")+1:].replace('"',"") for i in range(3)])
print(a_t)

 https://github.com/Abraxas-365/langchain-rust
 Коллеги, добрый день. А есть тут люди, занимающиеся консалтингом / заказной разработкой? Есть небольшой проект по подключению knowledge-base к LiveChat в качестве первой линии поддержки english / spanish / italian
 Да, обращайтесь. Сейчас примерно то же делаем для одного крупного ритейлера.


In [19]:
chain_sum = prompt | llm_sum | StrOutputParser()

In [27]:
chain_sum.invoke({'context': a_t})

'1. Коллеги ищут консультантов или разработчиков для проекта по подключению knowledge-base к LiveChat.\n2. Проект требует знание английского, испанского и итальянского языков.\n3. Один крупный ритейлер уже осуществляет подобное решение.'

In [20]:
start = time()
pre_sum_text = []
temp = []
temp_len = 0
numb_of_sum = 0
for _, text in messages[:50]:
    temp.append(text)
    temp_len += len(text)
    if len(temp) > 5 or temp_len > 3000:
        numb_of_sum +=1
        temp_str = '\n'.join(temp)
        sum_text = chain_sum.invoke({'context': temp_str})
        while sum_text.count("\n\n"):
            sum_text = sum_text.replace("\n\n", "\n")
        # print(sum_text)
        print(numb_of_sum, time()-start)
        print("-"*20)
        if len(sum_text) >20:
            pre_sum_text.append(sum_text)
        temp=[]
        temp_len = 0

if len(temp)!=0:
        numb_of_sum +=1
        temp_str = '\n'.join(temp)
        sum_text = chain_sum.invoke({'context': temp_str})
        while sum_text.count("\n\n"):
            sum_text = sum_text.replace("\n\n", "\n")
        # print(sum_text)
        print(numb_of_sum, time()-start)
        print("-"*20)
        if len(sum_text) >20:
            pre_sum_text.append(sum_text)
        temp=[]
        temp_len = 0

1 40.51199150085449
--------------------
2 65.16966533660889
--------------------
3 106.78874278068542
--------------------
4 133.9435052871704
--------------------
5 159.81736731529236
--------------------
6 183.54665350914001
--------------------
7 207.13014221191406
--------------------
8 234.34503650665283
--------------------
9 240.8223512172699
--------------------


In [21]:
print("Краткая выдержка")
for t in pre_sum_text:
    while t.count("\n\n"):
        t = t.replace("\n\n", "\n")
    print(t)

* L предоставляет ссылку на репозиторий langchain-rust.
* Roman N задает вопрос о консультации по консалтингу и заказной разработке для проекта LiveChat.
* Илья Нездешний отвечает, что они также работают над аналогичным проектом.
* Андрей Лебедев ищет помощи в решении задачи определения несоответствий чисел в тексте и таблицах PDF-документа.
* Rinat Abdullin предлагает использовать coze для быстрого прототипа.
* Денис Наумов ищет пример агентов для генерации новостей и контента для блогов.
* Сергей спрашивает о архитектуре.
* Леви (Юрий) Свердлов предоставляет ссылку на репозиторий gpt-newspaper.
* Ринат Абдуллин объясняет процесс самообучения модели и способы оценки качества системы.
* Сергей говорит о самообучении и рекомендует списки интересных решений.
* Денис Наумов выражает надежду на готовые решения и просит подсказки.
* Александр задает вопрос об улучшении системы с использованием данных о плохих ответах.
* Ринат Abdullin предлагает методы улучшения качества модели, такие как д

In [3]:
config = {"temperature": 0.7, "stop": ["\n\n\n"], "max_new_tokens": 512, "gpu_layers": 50, "top_k": 50, "top_p": 0.95, "context_length": 6000}
llm = CTransformers(model="/content/mistral-7b-openorca.Q5_K_M.gguf",model_type='mistral', config=config)

In [ ]:
# query = input()
# docs = db.similarity_search(query)
# context_search = "\n".join([doc.page_content for doc in docs])
# prompt_rag = f"""
# System:
# Используй следующие фрагменты контекста, чтобы в конце ответить на вопрос.
# Если ты не нашел ответа, просто скажи, что не знаешь ответа. Не пытайся выдумывать ответ.
# Используй максимум три предложения и старайся отвечать максимально кратко.
# {context_search}

# Вопрос: {query}
# Правильный ответ: """

# out = llm(prompt_rag)
# while out.count("\n\n"):
#         out = out.replace("\n\n", "\n")
# print(out)

 Недостатки Langchain



Макс) отвечтолько мой ответ: "Мый ответ, что я нет. Новая ответ ответ ответ ответ ответ ответ ответкакэтолько в какур"

















Ва вопроб)





















L Knownгмнетрандом"


In [11]:
template_rag = """Используй следующие фрагменты контекста, чтобы в конце ответить на вопрос.
Если ты не нашел ответа, просто скажи, что не знаешь ответа. Не пытайся выдумывать ответ.
Используй максимум три предложения и старайся отвечать максимально кратко.
{context}
Вопрос: {question}
Полезный ответ: """
prompt_rag = PromptTemplate.from_template(template_rag)

In [12]:
def format_docs(docs):
	return "\n\n".join(doc.page_content for doc in docs)

# Формируем конвеер
rag_chain = (
	{"context": db.as_retriever() | format_docs, "question": RunnablePassthrough()}
	| prompt_rag
	| llm
	| StrOutputParser()
)

query = input("Что вы хотите уточнить? ")

out = rag_chain.invoke(query)

print(out)

Что вы хотите уточнить? Недостатки Langchain
1. Документация сложна и не очень понятная 2. Волапай долго взаимодействовать с базой данных, что может привести к непосредственным проблемам 3. В langchain много интеграций, которые переписывать времени нет 4. Langchain требует большого объема дисковой памяти


